In [1]:
import json
from scraper import parsuj_misje_z_url
from sqlalchemy.exc import IntegrityError
from pathlib import Path

In [2]:
url = "https://warcraft.wiki.gg/wiki/Wrath_Unleashed_(quest)"
wynik = parsuj_misje_z_url(url)
wynik_json = json.dumps(wynik, ensure_ascii=False, indent=2)

In [3]:
print(wynik_json)

{
  "Źródło": {
    "url": "https://warcraft.wiki.gg/wiki/Wrath_Unleashed_(quest)"
  },
  "Misje_EN": {
    "Podsumowanie_EN": {
      "Tytuł": "Wrath Unleashed",
      "Start_NPC": "Turalyon",
      "Koniec_NPC": "Lady Liadrin",
      "Wymagany_Poziom": "80",
      "Doświadczenie": "1,750",
      "Nagrody": "23 40",
      "Poprzednia_Misja": "[80] Ethereal Eradication &  [80] Light's Arsenal",
      "Następna_Misja": "[80] Broken Sun"
    },
    "Cele_EN": {
      "Główny": {
        "1": "Join the charge of the Vanguard of Light."
      },
      "Podrzędny": {
        "1": "Aid Turalyon in attacking the Voidwake Behemoth",
        "2": "Channel the Light against the Voidwake Behemoth",
        "3": "Drive back Vorasius"
      }
    },
    "Treść_EN": {
      "1": "<Name>, the time has come! We push these fiends to Sun's Reach Harbor! We drive them back into their accursed Void portal and drive them from these sacred shores!",
      "2": "For the Light! For Azeroth!"
    },
    "Postę

In [4]:
from pomocnicze_funkcje_sql import *
from sqlalchemy import create_engine
from sqlalchemy.engine import URL, Engine
from sqlalchemy import text
from sqlalchemy.exc import IntegrityError

In [5]:
zapisz_npc_i_status_do_db_z_wyniku(
    silnik=utworz_engine_do_db(),
    tabela_npc="dbo.NPC",
    tabela_npc_statusy="dbo.NPC_STATUSY",
    szukaj_wg=["Start_NPC", "Koniec_NPC"],
    wyscrapowana_tresc=wynik,
    zrodlo="wiki",
    status="0_ORYGINAŁ"
)


In [7]:
pobierz_dane_z_db(
    silnik=utworz_engine_do_db(), 
    tabela="dbo.NPC_STATUSY"
    )

[{'TECH_ID': 1,
  'NPC_ID_FK': 1,
  'STATUS': '0_ORYGINAŁ',
  'NAZWA': 'Turalyon',
  'DATA_STATUS': datetime.datetime(2025, 12, 18, 22, 48, 27, 77280)},
 {'TECH_ID': 2,
  'NPC_ID_FK': 2,
  'STATUS': '0_ORYGINAŁ',
  'NAZWA': 'Lady Liadrin',
  'DATA_STATUS': datetime.datetime(2025, 12, 18, 22, 48, 27, 103885)}]